###**LTC or pressure datalogger correction for barometric pressure fluctuations**


---


**Connect to Shared Drive**

You will be prompted to click on a link that will show you an authorization code. copy the authorization code into the input box below. You also may be asked to allow Google Colab access to your Drive.

In [ ]:
#Connect to Drive, import packages
from google.colab import drive
drive.mount("/content/drive/")
import pandas as pd
import os

Specify file names for LTC and barometric files. Path name should be the same- make sure working files are in the preprocess_files folder.

In [ ]:
path='/content/drive/My Drive/Water/preprocess_files/'

#Rename the datfile and barofile to the filenames you want to use
datfile='MBM1_20210416_LTC.csv'
barofile='MB_20210416_baro.csv'

For **Solinst LTC loggers**, run code blocks #1-5 below.

Assumes input logger data file has the following columns: TEMPERATURE, LEVEL, CONDUCTIVITY. The units for the LEVEL column should be meters.

**#1. Imports custom function to find the start of the data in the logger file, opens and reads the logger file, opens and reads the barometric pressure file**

In [ ]:
#Import custom function - available for download on Github as find_first_row.py
import sys
sys.path.append('/content/drive/My Drive/CZN_hydro/Water/process_scripts')
import find_first_row
#Solinst typically have some logger metadata at the top of the file; this is to skip over that
ffr=find_first_row.first_row('/content/drive/My Drive/CZN_hydro/Water/preprocess_files/'+datfile)
ffrbaro=find_first_row.first_row('/content/drive/My Drive/CZN_hydro/Water/preprocess_files/'+barofile)
#print(ffr)

#Open data logger file
df=pd.read_csv(path+datfile,skiprows=ffr)
DateTime=df['Date']+' '+df['Time']
df.insert(0,'DateTime',DateTime)
df['DateTime']=pd.to_datetime(df['DateTime']).dt.round('15min').dt.strftime('%m/%d/%Y %H:%M:%S')
df=df.rename(columns={'LEVEL':'LOGGER_mWater','TEMPERATURE':'LOGGER_Temp_C','CONDUCTIVITY':'LOGGER_EC'})
df=df.set_index('DateTime')

#Bring in barometric data
barodf=pd.read_csv(path+barofile,skiprows=ffrbaro)
DateTime=barodf['Date']+' '+barodf['Time']
barodf.insert(0,'DateTime',DateTime)
barodf['DateTime']=pd.to_datetime(barodf['DateTime']).dt.round('15min').dt.strftime('%m/%d/%Y %H:%M:%S')
barodf=barodf.rename(columns={'LEVEL':'BARO_P_kPa','TEMPERATURE':'BARO_Temp_C'})
barodf['DateTime']=pd.to_datetime(barodf['DateTime'])
barodf=barodf.set_index('DateTime')

**#2. Merge the barometric data into the logger data**

In [ ]:
#Merge two dataframes, keeps ALL LTC records and interpolates any missing baro P values
df=df.merge(barodf['BARO_P_kPa'],how='left',left_index=True,right_index=True)
df['BARO_P_kPa']=df['BARO_P_kPa'].interpolate(axis=0).fillna(method='bfill')

**#3. Convert barometric pressure to meters of water, correct water levels for barometric pressure**

In [ ]:
#Conversion constants
PSI_PSF=144
KPA_PSI=0.1450377
FT_M=0.3048
PSI_FT=2.0366

#Convert baro pressure to meters water
df.rename(columns = {'BARO_P_kPa':'BARO_mWater'}, inplace=True)
df['BARO_mWater']=(df['BARO_mWater']*KPA_PSI*PSI_FT*FT_M)
BP1=df['BARO_mWater'][0]

#Correct for baro pressure
df['LOGGER_mWater_corr']=df['LOGGER_mWater']+(BP1-df['BARO_mWater'])

#Previous beginning of new dataframe
df.head()

**#4. Convert logger-measured EC to SC**

In [ ]:
#Convert EC to SC
EC=df['LOGGER_EC']
TEMP=df['LOGGER_Temp_C']
df['LOGGER_SC']=round(EC/(1+0.0191*(TEMP-25)),1)

**#5. Saves the corrected data to the same folder with a new suffix ('_baroC') and deleted the old copy**

In [ ]:
#Save a copy to the folder with a suffix and delete old copy
df.to_csv(path+datfile.split('.')[0]+'_baroC.csv')
os.remove(path+datfile)